<a href="https://colab.research.google.com/github/fdhernandezgutierrez/LUNET_Brain-Tumor/blob/main/nnUNet_test_on_BRaTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install triton
!git clone https://github.com/MIC-DKFZ/nnUNet.git
!git clone https://github.com/NVIDIA/apex
!pip install -e ./nnUNet
!pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

In [ ]:
import os
import shutil
import random
import multiprocessing
from multiprocessing import Pool
import SimpleITK as sitk
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *
from nnunetv2.dataset_conversion.generate_dataset_json import generate_dataset_json

In [ ]:
%%capture
!mkdir -p /content/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/Dataset101_BraTS2021
!mkdir -p /content/brats2021/training
%cd /content/brats2021/training
!tar xvf /content/drive/MyDrive/Colab\ Notebooks/datafolder/BRATS2021/BraTS2021_Training_Data.tar
os.environ['nnUNet_raw'] = "/content/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/"
os.environ['nnUNet_results'] = '/content/RESULTS_FOLDER'
os.environ['nnUNet_preprocessed'] = '/content/nnUNet_preprocessed'

In [ ]:
from nnunetv2.paths import nnUNet_raw

In [ ]:
def copy_BraTS_segmentation_and_convert_labels_to_nnUNet(in_file: str, out_file: str) -> None:
    # use this for segmentation only!!!
    # nnUNet wants the labels to be continuous. BraTS is 0, 1, 2, 4 -> we make that into 0, 1, 2, 3
    img = sitk.ReadImage(in_file)
    img_npy = sitk.GetArrayFromImage(img)

    uniques = np.unique(img_npy)
    for u in uniques:
        if u not in [0, 1, 2, 4]:
            raise RuntimeError('unexpected label')

    seg_new = np.zeros_like(img_npy)
    seg_new[img_npy == 4] = 3
    seg_new[img_npy == 2] = 1
    seg_new[img_npy == 1] = 2
    img_corr = sitk.GetImageFromArray(seg_new)
    img_corr.CopyInformation(img)
    sitk.WriteImage(img_corr, out_file)


def convert_labels_back_to_BraTS(seg: np.ndarray):
    new_seg = np.zeros_like(seg)
    new_seg[seg == 1] = 2
    new_seg[seg == 3] = 4
    new_seg[seg == 2] = 1
    return new_seg


def load_convert_labels_back_to_BraTS(filename, input_folder, output_folder):
    a = sitk.ReadImage(join(input_folder, filename))
    b = sitk.GetArrayFromImage(a)
    c = convert_labels_back_to_BraTS(b)
    d = sitk.GetImageFromArray(c)
    d.CopyInformation(a)
    sitk.WriteImage(d, join(output_folder, filename))


def convert_folder_with_preds_back_to_BraTS_labeling_convention(input_folder: str, output_folder: str, num_processes: int = 12):
    """
    reads all prediction files (nifti) in the input folder, converts the labels back to BraTS convention and saves the
    """
    maybe_mkdir_p(output_folder)
    nii = subfiles(input_folder, suffix='.nii.gz', join=False)
    with multiprocessing.get_context("spawn").Pool(num_processes) as p:
        p.starmap(load_convert_labels_back_to_BraTS, zip(nii, [input_folder] * len(nii), [output_folder] * len(nii)))

In [ ]:
brats_data_dir = '/content/brats2021/training'

task_id = 101
task_name = "BraTS2021"

foldername = "Dataset%03.0d_%s" % (task_id, task_name)

# setting up nnU-Net folders
out_base = join(nnUNet_raw, foldername)
imagestr = join(out_base, "imagesTr")
labelstr = join(out_base, "labelsTr")
imagests = join(out_base, "imagesTs")
labelsts = join(out_base, "labelsTs")
maybe_mkdir_p(imagestr)
maybe_mkdir_p(labelstr)
maybe_mkdir_p(imagests)
maybe_mkdir_p(labelsts)

case_ids = subdirs(brats_data_dir, prefix='BraTS', join=False)

random.shuffle(case_ids)

train_percentage = 0.8
train_size = int(len(case_ids) * train_percentage)

for i, c in enumerate(case_ids):
    if i < train_size:
      shutil.copy(join(brats_data_dir, c, c + "_t1.nii.gz"), join(imagestr, c + '_0000.nii.gz'))
      shutil.copy(join(brats_data_dir, c, c + "_t1ce.nii.gz"), join(imagestr, c + '_0001.nii.gz'))
      shutil.copy(join(brats_data_dir, c, c + "_t2.nii.gz"), join(imagestr, c + '_0002.nii.gz'))
      shutil.copy(join(brats_data_dir, c, c + "_flair.nii.gz"), join(imagestr, c + '_0003.nii.gz'))

      copy_BraTS_segmentation_and_convert_labels_to_nnUNet(join(brats_data_dir, c, c + "_seg.nii.gz"),
                                                          join(labelstr, c + '.nii.gz'))
    else:
      shutil.copy(join(brats_data_dir, c, c + "_t1.nii.gz"), join(imagests, c + '_0000.nii.gz'))
      shutil.copy(join(brats_data_dir, c, c + "_t1ce.nii.gz"), join(imagests, c + '_0001.nii.gz'))
      shutil.copy(join(brats_data_dir, c, c + "_t2.nii.gz"), join(imagests, c + '_0002.nii.gz'))
      shutil.copy(join(brats_data_dir, c, c + "_flair.nii.gz"), join(imagests, c + '_0003.nii.gz'))

      copy_BraTS_segmentation_and_convert_labels_to_nnUNet(join(brats_data_dir, c, c + "_seg.nii.gz"),
                                                          join(labelsts, c + '.nii.gz'))
generate_dataset_json(out_base,
                      channel_names={0: 'T1', 1: 'T1ce', 2: 'T2', 3: 'Flair'},
                      labels={
                          'background': 0,
                          'whole tumor': (1, 2, 3),
                          'tumor core': (2, 3),
                          'enhancing tumor': (3, )
                      },
                      num_training_cases = train_size,
                      file_ending='.nii.gz',
                      regions_class_order=(1, 2, 3),
                      license='see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863',
                      reference='see https://www.synapse.org/#!Synapse:syn25829067/wiki/610863',
                      dataset_release='1.0')

In [ ]:
!cp -r /content/brats2021dataset /content/drive/MyDrive/Colab\ Notebooks/datafolder

In [ ]:
os.environ['nnUNet_raw'] = "/content/drive/MyDrive/Colab Notebooks/datafolder/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/"
os.environ['nnUNet_results'] = '/content/RESULTS_FOLDER'
os.environ['nnUNet_preprocessed'] = '/content/nnUNet_preprocessed'
!mkdir -p /content/RESULTS_FOLDER
!mkdir -p /content/nnUNet_preprocessed

In [ ]:
!nnUNetv2_plan_and_preprocess -d 101 --verify_dataset_integrity

Fingerprint extraction...
Dataset101_BraTS2021
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 1000/1000 [19:02<00:00,  1.14s/it]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [140. 171. 137.], 3d_lowres: [140, 171, 137]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 105, 'patch_size': (1

In [ ]:
!cp -r /content/nnUNet_preprocessed /content/drive/MyDrive/Colab\ Notebooks/datafolder

In [ ]:
os.environ['nnUNet_raw'] = "/content/drive/MyDrive/Colab Notebooks/datafolder/brats2021dataset/nnUNet_raw_data_base/nnUNet_raw_data/"
os.environ['nnUNet_results'] = '/content/RESULTS_FOLDER'
os.environ['nnUNet_preprocessed'] = '/content/drive/MyDrive/Colab Notebooks/datafolder/nnUNet_preprocessed'
!mkdir -p /content/RESULTS_FOLDER
!mkdir -p /content/nnUNet_preprocessed

In [ ]:
!nnUNetv2_train 101 2d 0 --npz


############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0
/content/nnUNet/nnunetv2/training/nnUNetTrainer/nnUNetTrainer.py:164: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.grad_scaler = GradScaler() if self.device.type == 'cuda' else None

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2024-09-22 07:40:11.64166